# Table of contents
1. [Requirements](#Requirements)
2. [Introduction](#Introduction)
3. [Imports](#Imports)
    1. [Libraries](#Libraries)
    2. [Data](#Data)
4. [Data Exploration](#data-exploration)
5. [Feature Engineering](#feature-engineering)
6. [Modelling](#modelling)
    1. [Baseline](#baseline)
    2. [LSTM](#lstm)
7. [Results Analysis](#results-analysis)

# Requirements

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 25.6 MB/s 


## Introduction

## Imports

### Libraries


In [2]:
import kerastuner as kt
import numpy as np
import pandas as pd
import sympy
import tensorflow as tf
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from kerastuner.engine.hyperparameters import HyperParameters
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler

<ipython-input-2-05635ca4cbb8>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


### Data

In [3]:
# Mount the drive
drive.mount("/content/drive")

# Load the data
path = "/content/drive/MyDrive/IMS DLNN/Projeto/" # Deve apontar para as pastas do dataset na drive

hourly_energy_consumption = pd.read_csv(path + "consumption.csv")
hourly_weather = pd.read_csv(path + "weather.csv")



Mounted at /content/drive


## Data Exploration

In [4]:
hourly_energy_consumption.head()

,utc_timestamp,cet_cest_timestamp,PT_load_actual_entsoe_transparency,PT_load_forecast_entsoe_transparency,PT_solar_generation_actual,PT_wind_generation_actual,PT_wind_offshore_generation_actual,PT_wind_onshore_generation_actual
0,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01T01:00:00Z,2015-01-01T02:00:00+0100,5123.9,4820.0,NaN,NaN,NaN,551.0
2,2015-01-01T02:00:00Z,2015-01-01T03:00:00+0100,4771.1,4521.0,NaN,NaN,NaN,596.5
3,2015-01-01T03:00:00Z,2015-01-01T04:00:00+0100,4443.5,4250.0,NaN,NaN,NaN,706.3
4,2015-01-01T04:00:00Z,2015-01-01T05:00:00+0100,4234.9,4083.0,NaN,NaN,NaN,720.5


## Feature engineering

In [5]:
hourly_energy_consumption["utc_timestamp"] = pd.to_datetime(hourly_energy_consumption["utc_timestamp"])

# As seguintes colunas ou estão todas NaN, ou não são relevantes para o modelo
hourly_energy_consumption.drop([
    "cet_cest_timestamp",
    "PT_wind_generation_actual",
    "PT_wind_offshore_generation_actual",
    "PT_wind_onshore_generation_actual",
    "PT_load_forecast_entsoe_transparency",
    "PT_solar_generation_actual",
], axis=1, inplace=True)

# Os consumos em falta serão preenchidos com os valores seguintes, no pior dos cenários
# nunca serão superiores a estes, tendo em conta que os valores que faltam são
# periodos da noite, a alteração não será significativa
hourly_energy_consumption["PT_load_actual_entsoe_transparency"] = hourly_energy_consumption["PT_load_actual_entsoe_transparency"].bfill()

# Criamos os dados referentes a hora, dia da semana, dia e mês, para que o modelo
# aprenda a sazonalidade do consumo, dias mais ativos e menos ativos
hourly_energy_consumption["hour"] = hourly_energy_consumption["utc_timestamp"].dt.hour
hourly_energy_consumption["day_of_week"] = hourly_energy_consumption["utc_timestamp"].dt.dayofweek
hourly_energy_consumption["month"] = hourly_energy_consumption["utc_timestamp"].dt.month
hourly_energy_consumption["day"] = hourly_energy_consumption["utc_timestamp"].dt.dayofyear

In [6]:
hourly_energy_consumption.shape

(43823, 6)

In [7]:
hourly_weather["utc_timestamp"] = pd.to_datetime(hourly_weather["utc_timestamp"])

hourly_weather.drop([
    "PT_radiation_direct_horizontal",
    "PT_radiation_diffuse_horizontal"
], axis=1, inplace=True)

In [8]:
hourly_weather.shape

(43823, 2)

In [9]:
def create_data_window(
    data,
    past_steps,
    future_steps,
    batch_size=1
):
  """
  Create a data windowed dataset based on the received numpy data array
  """
  window_len = past_steps + future_steps

  # Criamos um tensor dataset via numpy array e obtemos a janela de dados
  dataset = tf.data.Dataset.from_tensor_slices(data)
  dataset = dataset.window(window_len, shift=future_steps, drop_remainder=True)

  # Achata-se o dataset para a nossa janela
  dataset = dataset.flat_map(lambda x : x.batch(window_len))
  dataset = dataset.map(lambda x : (x[:-future_steps], x[-future_steps:, :1]))

  return dataset.batch(batch_size).prefetch(1)


In [10]:
def split_dataset(
    data,
    train_val_ratio=0.70,
    test_size=8760
):
  """
  Split the data into 3 datasets (train, validation and test)
  
  First remove the last N entries (test_size) for test dataset (default one year)
  and the remaining is split according to the ratio received (train_val_ration)
  into train and validation datasets
  """
  
  # Primeiro passo: partimos o dataset em 2 partes, as últimas N entradas
  # serão para o dataset de teste, o restante será treino/validação
  test_index = len(data) - test_size
  test_data = data[test_index:]
  remaining_data = data[:test_index]

  # Segundo passo: dividimos o remanescente em 2 datasets segundo o rácio
  val_index = int(len(remaining_data) * train_val_ratio)
  train_data = remaining_data[:val_index]
  validation_data = remaining_data[val_index:]

  return train_data, validation_data, test_data
  


Porquê o uso de tensor datasets em vez do tradicional X_<train/test> e y_<train/test>?

O tensor dataset já possui essa informação, assim apenas é necessário passar apenas este bloco e o modelo já sabe onde está o input e output

In [11]:
train_val_ratio = 0.7
past_steps = 24*7 # A nossa estimativa será efetuada com base nos 7 dias anteriores
timesteps = 24 # Queremos prever as próximas 24H
batch_size = 120
features = 5

In [12]:
def get_datasets(
    energy_consumption,
    weather
):
  """
  Creates a prepared tensor datasets (train/test/val) ready to be used
  by the neural network models based on the hourly consumption and weather dataframes
  """
  scaler = MinMaxScaler()

  # Juntamos os 2 dataframes usando o timestamp como elo de ligação
  # e por fim descartamos o timestamp, pois não terá influência no modelo 
  final_df = pd.merge(energy_consumption, weather, on="utc_timestamp")
  final_df.drop("utc_timestamp", axis=1, inplace=True)

  final_df = scaler.fit_transform(final_df)

  # Dividimos os dados em 3 datasets
  train_data, val_data, test_data = split_dataset(
      data=final_df, 
      train_val_ratio=train_val_ratio,
  )

  # Criamos tensor datasets que serão usados para alimentar os modelos
  train_ds = create_data_window(train_data, past_steps, timesteps, batch_size)
  val_ds = create_data_window(val_data, past_steps, timesteps, batch_size)
  test_ds = create_data_window(test_data, past_steps, timesteps, batch_size)

  return train_ds, val_ds, test_ds


In [13]:
train_ds, val_ds, test_ds = get_datasets(
    hourly_energy_consumption,
    hourly_weather
)

In [14]:
for idx,(x,y) in enumerate(train_ds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    break

x =  (120, 168, 6)
y =  (120, 24, 1)


In [19]:
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>

## Modelling

In [15]:
# Aqui tratar de qualquer operação ou criação de variáveis que sejam
# necessárias para o processo de modelação de DL.
# Number of samples
n_samples = len(hourly_energy_consumption)

# Number of time steps
n_timesteps = 24
# Number of features
n_features = 3
# Reshape the data into a 3D array to feed the Neural Netwokrs
# X = np.empty((n_samples, n_timesteps, n_features))
# X[:, :, 0] = hourly_energy_consumption
# X[:, :, 1] = hourly_temperature
# X[:, :, 2] = hourly_radiation

### Baseline
        Persistence

A persistência é o método de baseline mais condiserado e que, como o nome indica, considera que o valor para o futuro é igual à ultima observação. Pode ser denotado pela seguinte equação:

$ T_{t+1} = T_{t} $ 

In [18]:
# TODO

# creating the persistence matrix
# df.merged.shape should be Amount of predictions we want to gather X 24
# persistence_forecasts=np.zeros((dfmerged.shape),dtype=float)
# for i in range(len(dfmerged)):
#     persistence_forecasts.iloc[i,:]=dfmerged.iloc[i,-1]

### LSTM

In [17]:
# Define the model
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32), input_shape=(timesteps, features)))
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(24))
    
    # Choose an optimizer
    optimizer = hp.Choice('optimizer', ['adam', 'sgd','rmsprop'])
    if optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
    elif optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
    else:    
        optimizer = tf.keras.optimizers.RMSprop(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
        
    # Compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model
# Define the search space for Keras Tuner
hps = HyperParameters()
hps.Choice('batch_size', [32, 64, 128, 256])
hps.Choice('activation', ['relu', 'tanh','sigmoid'])
# Use the Keras Tuner to search for the best set of hyperparameters
tuner = kt.Hyperband(
    hypermodel=build_model, 
    objective="val_accuracy",
    hyperparameters=hps
)
tuner.search(train_ds, epochs=100,batch_size=1, validation_data=val_ds)
# Get the best model from the search
best_model = tuner.get_best_model()
# Use the best model to make predictions on the test set
y_pred = best_model.predict(test_ds)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
32                |?                 |batch_size
relu              |?                 |activation
224               |?                 |units
0.2               |?                 |dropout
sgd               |?                 |optimizer
0.0073129         |?                 |learning_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
4                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2


ValueError: ignored

## Results Analysis